In [1]:
%pip install --quiet pypdf langchain_experimental sentence_transformers langchain-huggingface pymupdf

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -qU "langchain-chroma>=0.1.2"

In [2]:
import torch
if torch.cuda.is_available():
    print('GPU disponível')

GPU disponível


In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(r"C:\Users\erick\Downloads\Gerenciamento e qualidade de software.pdf")
pages = loader.load_and_split()

In [4]:
pages[0].page_content

'Disciplina\nGerenciamento e Qualidade de\nSoftware\nUnidade 1\nGerenciamento e qualidade de software\nAula 1\nIntrodução ao gerenciamento de software\nIntrodução\nOlá, estudante!\xa0\xa0\nSoftware desempenha um papel cada vez mais importante no cotidiano das pessoas, sendo\namplamente utilizado em diversas atividades, como ouvir música, assistir a filmes e se\ncomunicar com os amigos. Além disso, ele impulsiona a transformação digital, tal como o uso de\ninteligência artificial, por exemplo, o ChatGPT e o aprendizado de máquina baseado em\nrecomendações.\xa0\nNo entanto, a complexidade do software pode representar um desafio, pois envolve dificuldades\nno entendimento e no desenvolvimento de sistemas complexos, principalmente quando há\nmúltiplas pessoas trabalhando ao longo de um período no mesmo sistema.'

In [5]:
import re
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(r"C:\Users\erick\Downloads\Gerenciamento e qualidade de software.pdf")
pages = loader.load_and_split()

def normalize_text(text):
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'Disciplina Gerenciamento e Qualidade de Software+', ' ', text)
    text = text.strip()
    return text

# Normalizar o conteúdo de cada página
for page in pages:
    page.page_content = normalize_text(page.page_content)

# Exibir o conteúdo da primeira página após a normalização
pages[0].page_content

'Unidade 1 Gerenciamento e qualidade de software Aula 1 Introdução ao gerenciamento de software Introdução Olá, estudante! Software desempenha um papel cada vez mais importante no cotidiano das pessoas, sendo amplamente utilizado em diversas atividades, como ouvir música, assistir a filmes e se comunicar com os amigos. Além disso, ele impulsiona a transformação digital, tal como o uso de inteligência artificial, por exemplo, o ChatGPT e o aprendizado de máquina baseado em recomendações. No entanto, a complexidade do software pode representar um desafio, pois envolve dificuldades no entendimento e no desenvolvimento de sistemas complexos, principalmente quando há múltiplas pessoas trabalhando ao longo de um período no mesmo sistema.'

In [6]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings


# model_name = "BAAI/bge-m3"
model_name = "paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": False}


embedding_model = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

text_splitter = SemanticChunker(embedding_model)

# Iterar sobre cada página e criar chunks semânticos para cada uma
all_chunks = []
for page in pages:
    # Criar os chunks para o conteúdo da página
    chunks = text_splitter.create_documents([page.page_content])
    all_chunks.extend(chunks)

# Exibir o conteúdo do primeiro chunk da primeira página
print(all_chunks[0].page_content)

c:\Users\erick\anaconda3\envs\cuda_env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\erick\anaconda3\envs\cuda_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\erick\anaconda3\envs\cuda_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted 

Unidade 1 Gerenciamento e qualidade de software Aula 1 Introdução ao gerenciamento de software Introdução Olá, estudante! Software desempenha um papel cada vez mais importante no cotidiano das pessoas, sendo amplamente utilizado em diversas atividades, como ouvir música, assistir a filmes e se comunicar com os amigos.


In [7]:
from langchain_chroma import Chroma

db = Chroma(persist_directory = './', embedding_function = embedding_model)


In [8]:
db.add_documents(all_chunks)

: 

In [16]:
db.similarity_search('O que é qualidade de software?')[0].page_content

'Boston: McGraw-Hill, 2014. Aula 2 Introdução de teste de software Introdução Disciplina Gerenciamento e Qualidade de Software Olá, estudante! Já sabemos o quão importante é o desenvolvimento de software. Dessa forma, não é estranho perceber que a qualidade é um fator crítico que permeia tudo aquilo dos quais somos dependentes. Imagine depender de um sistema de comunicação, entretenimento ou transações financeiras, e esse sistema não funcionar corretamente. Com o intuito de atender às exigências de maior qualidade, desenvolvedores se motivaram a criar métodos e técnicas para o software atingir padrões de qualidade exigidos. Tais exigências não eram injustificadas, pois a humanidade entrou em um período em que a tecnologia afetou significativamente a vida coletiva. No que diz respeito à saúde, às finanças, ao transporte, à educação e à segurança, a menor falha pode ser responsável por uma catástrofe. É a partir desse tipo de cenário que os testes de software assumem um papel fundamental

In [ ]:
'Boston: McGraw-Hill, 2014. Aula 2 Introdução de teste de software Introdução Disciplina Gerenciamento e Qualidade de Software Olá, estudante! Já sabemos o quão importante é o desenvolvimento de software. Dessa forma, não é estranho perceber que a qualidade é um fator crítico que permeia tudo aquilo dos quais somos dependentes. Imagine depender de um sistema de comunicação, entretenimento ou transações financeiras, e esse sistema não funcionar corretamente. Com o intuito de atender às exigências de maior qualidade, desenvolvedores se motivaram a criar métodos e técnicas para o software atingir padrões de qualidade exigidos. Tais exigências não eram injustificadas, pois a humanidade entrou em um período em que a tecnologia afetou significativamente a vida coletiva. No que diz respeito à saúde, às finanças, ao transporte, à educação e à segurança, a menor falha pode ser responsável por uma catástrofe. É a partir desse tipo de cenário que os testes de software assumem um papel fundamental. Nesta aula, abordaremos conceitos de qualidade e teste de software, objetivos de testar e os principais tipos de teste. Vamos lá? Introdução ao teste de software Disciplina Gerenciamento e Qualidade de Software Quando a humanidade se deu conta de que o software estava integrado na maior parte da vida cotidiana de pessoas, surgiu uma corrida pela maior qualidade de software. Qualidade não era uma preocupação no início da era digital. Hoje, todos os desenvolvedores de software concordam que software de alta qualidade é um objetivo importante. Definir qualidade de software não é uma tarefa fácil. Para Pressman e Maxim (2021), no sentido mais geral, é uma gestão de qualidade para criar um produto útil que forneça valor mensurável para aqueles que o produzem e para aqueles que o utilizam. Já para Sommerville (2011), a qualidade do software é diretamente relacionada à qualidade do processo de desenvolvimento de software. Mas, e qualidade? Segundo o dicionário, qualidade é a propriedade positiva de um objeto ou um ser (Aulete, 2009). A qualidade de software e os testes de software estão intrinsecamente relacionados, pois os testes desempenham um papel fundamental na busca por qualidade no desenvolvimento de software. Isso quer dizer que testar software é avaliar se ele está fazendo o que deveria fazer, de acordo com os seus requisitos, e não está fazendo o que não deveria fazer (Moreira Filho; Rios, 2003). Testar também pode ser um processo de executar um programa ou sistema com a intenção de encontrar defeitos (teste negativo) (Myers, 1979). Para Hetzel (1988), teste é qualquer atividade que a partir da avaliação de um atributo ou capacidade de um programa ou sistema seja possível determinar se ele alcança os resultados desejados. E para Pressman e Maxim (2021), é um conjunto de atividades que podem ser planejadas com antecedência e executadas sistematicamente. Todas essas definições realçam a seguinte ideia: teste de software é a verificação feita sobre um sistema ou parte dele para garantir que uma determinada entrada produza sempre uma saída esperada. Disciplina Gerenciamento e Qualidade de Software Dessa forma, compreendemos que, no âmbito do desenvolvimento de software, qualidade é compreendida como um conjunto de condições que devem ser satisfeitas ou refere-se à medida em que o software atende aos requisitos, sendo livre de defeitos, seguro, confiável e que atenda às necessidades do usuário. Independentemente do projeto que se desenvolve, existem vários objetivos pelos quais devemos testar software. O CTFL Syllabus (2023), em sua versão 4.0, destaca os seguintes: Avaliar produtos de trabalho, como requisitos, histórias de usuários, projetos e código. Detectar falhas e defeitos. Garantir a cobertura necessária de um objeto de teste. Reduzir o nível de risco de qualidade de software inadequado. Verificar se os requisitos especificados foram atendidos. Verificar se um objeto de teste está em conformidade com os requisitos contratuais, legais e normativos. Fornecer informações aos stakeholders para que possam tomar decisões informadas. Criar confiança na qualidade do objeto de teste. Validar se o objeto de teste está completo e funciona conforme o esperado pelos stakeholders. Os objetivos dos testes podem variar, dependendo do contexto, o que inclui o produto de trabalho que está sendo testado, o nível de teste, os riscos, o ciclo de vida de desenvolvimento de software que está sendo seguido e os fatores relacionados ao contexto do negócio (CTFL Syllabus, 2023). Explorando os níveis de teste de software Disciplina Gerenciamento e Qualidade de Software Já sabemos os conceitos de qualidade e teste de software e compreendemos o quão importante é garantir que o software atenda aos requisitos estabelecidos, sendo livre de defeitos e que entregue valor aos seus desenvolvedores e usuários. Mas, quando falamos em teste de software, imediatamente deve surgir a seguinte dúvida: quando testar? Nesse ponto, estamos falando em níveis de teste, os quais são grupos de atividades de teste que são organizadas e gerenciadas em conjunto. Cada nível de teste é uma instância do processo de teste, realizado em relação ao software em um determinado estágio de desenvolvimento, desde componentes individuais até sistemas completos (CTFL Syllabus, 2023). Existem sete níveis de teste de software, que são realizados em momentos diferentes do ciclo de vida do desenvolvimento de um software. Os níveis de teste são: Teste de Unidade: verifica o funcionamento do menor componente do software, como sub- rotinas, métodos e classes. É realizado pelo desenvolvedor e, geralmente, requer o uso de estruturas de teste ou frameworks de teste de unidade. Teste de Integração: verifica se a interação entre os componentes de um sistema é eficaz e não causa conflitos. É realizado pelo desenvolvedor e envolve a integração entre um ou mais componentes. Teste de Sistema: verifica o sistema como um todo, analisando o comportamento geral e seus recursos. É realizado por uma equipe de testes após a codificação completa do sistema. É realizado para verificar se o sistema atende aos requisitos definidos. Teste de Aceitação: verifica o sistema como um todo, sob o ponto de vista do usuário final, concentrando-se na validação dos requisitos. É realizado pelo usuário. O foco é verificar se o sistema está pronto para ser entregue e usado. Disciplina Gerenciamento e Qualidade de Software Teste Alfa: verifica o sistema de uma forma que não tenha sido planejada, sob o ponto de vista de um seleto grupo de usuários internos. É realizado pelos usuários internos da organização, podendo incluir testadores, desenvolvedores e outros funcionários. Teste Beta: verifica o sistema de uma forma que não tenha sido planejada, sob o ponto de vista de um grande número de usuários. É realizado por um subconjunto de usuários finais do sistema, que satisfaçam determinados critérios definidos pelo fornecedor do sistema. Teste de Regressão: verifica o sistema após alterações, como correções de bugs ou implementação de novas funcionalidades. É realizado pela equipe de testes. Os níveis de teste são diferenciados pelos atributos que lhes convêm, para evitar que as atividades de testes se repitam. O CTFL Syllabus (2023) destaca os seguintes: Objeto de teste. Objetivos do teste. Base de teste. Defeitos e falhas. Abordagem e responsabilidades. Cada um desses atributos ajuda a caracterizar e diferenciar os diferentes níveis de teste, garantindo que cada fase de teste tenha um foco específico e contribua para a qualidade geral do software. Compreender esses atributos permite que os profissionais de teste planejem e executem testes de maneira mais eficaz, atingindo os objetivos definidos em cada etapa do processo de desenvolvimento de software. Níveis de teste na prática Disciplina Gerenciamento e Qualidade de Software Agora que somos capazes de compreender a importância dos níveis de teste, no desenvolvimento de aplicações, nosso objetivo é entender como esses níveis de testes são executados em cada etapa do processo de desenvolvimento de software. Os níveis de teste são agrupamentos de atividades de teste bem planejadas e executadas de maneira organizada. Cada nível de teste representa uma fase do processo de teste. Cada nível de teste contribui para a melhoria geral da qualidade do software, conforme definido pelo CTFL Syllabus (2023). Em um ciclo de vida de desenvolvimento, existem sete níveis distintos de teste de software, cada um sendo executado em um momento específico. Segue-se a ordem: Teste de Unidade: é a menor parte testável do sistema. Nesse nível, o foco está nas partes mínimas do software, como funções e métodos, verificando se eles funcionam individualmente. Veja a Figura 1, “Função soma”, escrita em python. Disciplina Gerenciamento e Qualidade de Software Figura 1 | Função soma. Fonte: elaborada pelo autor. Nesse exemplo, o resultado da soma de 5 e 3 deve ser sempre 8. Este teste é muito importante para garantir que o “alicerce” do sistema funcione conforme o esperado. Teste de Integração: ocorre quando os componentes individuais do sistema são combinados. Isso garante que a interação entre esses componentes não resulte em conflitos. Este teste é uma atividade realizada pelos desenvolvedores, em que eles unem um ou mais componentes para verificar a integração. Veja os exemplos de integração a seguir: Figura 2 | Módulo formatador. Fonte: elaborada pelo autor. Disciplina Gerenciamento e Qualidade de Software Figura 3 | Módulo operações. Fonte: elaborada pelo autor. Figura 4 | Módulo de integração. Fonte: elaborada pelo autor. Nesses exemplos, o módulo de integração verifica se a função de formatação cria a saída esperada com base no resultado da função de soma. Dessa forma, garantimos que as partes do software interajam sem problemas, como aconteceria em uso real. Teste de Sistema: por sua vez, avalia o sistema completo, analisando seu comportamento geral e recursos. Uma equipe de testes executa esse nível após a codificação completa do sistema. O objetivo é garantir que o sistema atenda aos requisitos definidos, funcionando como uma unidade coesa. Sobre a perspectiva do usuário final, temos o Teste de Aceitação. Nesse nível, os requisitos são validados em relação à expectativa do usuário. O teste também é executado a nível de sistema e pelo próprio usuário final. Ele determina se o sistema está válido, ou seja, pronto para uso. Os testes Alfa e Beta são voltados para avaliações não planejadas do sistema. O Teste Alfa é semelhante ao Teste de Aceitação, porém neste nível apenas um grupo seleto é incluído para testadores e desenvolvedores, verificando o sistema de maneira imprevista. O objetivo é garantir um maior nível de qualidade do produto antes de enviá-lo ao cliente, pois permite aos desenvolvedores resolver imediatamente problemas críticos ou correções identificadas. Enquanto isso, no Teste Beta, um número maior de usuários finais avalia o sistema em condições do mundo real, proporcionando feedback valioso. Este teste coleta a opinião dos usuários sobre o produto e garante que o produto esteja pronto para usuários em tempo real. Por fim, o Teste de Regressão entra em ação após modificações no software, como correção de bugs ou adição de funcionalidades. É uma verificação para garantir que as mudanças não Disciplina Gerenciamento e Qualidade de Software impactem negativamente o sistema. Portanto, compreender a dinâmica dos níveis de teste ajuda os profissionais de teste a planejar e executar testes de forma eficaz. Cada nível tem seu propósito único, mas todos são essenciais para entregar um software confiável, livre de defeitos e que atenda às necessidades dos usuários. Videoaula: Introdução de teste de software Este conteúdo é um vídeo! Para assistir este conteúdo é necessário que você acesse o AVA pelo computador ou pelo aplicativo. Você pode baixar os vídeos direto no aplicativo para assistir mesmo sem conexão à internet. Olá, estudante! Nesta aula, você compreendeu os conceitos de qualidade de software, teste de software e os níveis de teste de software, além dos motivos pelos quais eles são importantes. Neste vídeo, consolidaremos esses conceitos de forma que as ideias sobre eles fiquem mais claras. Saiba mais Disciplina Gerenciamento e Qualidade de Software Para expandir seus conhecimentos sobre processos de software e testes de software, aconselhamos a leitura da obra indicada a seguir, com o objetivo de consolidar seus conhecimentos. Processos de software, de Polyanna Fabris e Luis Perini: neste livro, os autores discorrem sobre modelagem de processos, o que é fundamental para compreender a necessidade que se tem de parâmetros das métricas e revisões de software. Informações sobre engenharia de requisitos e o gerenciamento de projetos também são apresentadas. Referências Disciplina Gerenciamento e Qualidade de Software AULETE, C. Minidicionário Contemporâneo da Língua Portuguesa. 2. ed. Rio de Janeiro: Lexikon, 2009. CTFL SYLLABUS.'